In [ ]:
# !pip install pytube
# !pip install ffmpeg moviepy
# !pip install SpeechRecognition
# !pip install pydub

In [ ]:
# importing the module 
from pytube import YouTube 
from moviepy.editor import VideoFileClip
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence


# link of the video to be downloaded 
link = "your youtube video link"

def convert_video_to_audio_moviepy(video_file, output_ext="wav"):
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    try:
        filename, ext = os.path.splitext(video_file)
        clip = VideoFileClip(video_file)
        clip.audio.write_audiofile(f"{filename}.{output_ext}")
    except:
        return "Some Error!"
    return "Task Completed!"

def yt_download_video(link):
    try: 
        # object creation using YouTube 
        # which was imported in the beginning 
        yt = YouTube(link) 
    except: 
        return "Connection Error" #to handle exception 

    try:
        yt.streams.filter(progressive = True, file_extension = "mp4").first().download(filename = link.split("=")[-1]+".webm")
    #     yt.streams.filter(progressive = True,file_extension = "mp4").first().download(output_path = "E:\folder_name", filename = "Reiner and Bertholdt Transformation scene")
    except:
        return "Some Error!"
    return "Task Completed!"

# create a speech recognition object
r = sr.Recognizer()

# a function to recognize speech in the audio file
# so that we don't repeat ourselves in in other functions
def transcribe_audio(path):
    # use the audio file as the audio source
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        # try converting it to text
        text = r.recognize_google(audio_listened)
    return text

# a function that splits the audio file into chunks on silence
# and applies speech recognition
def get_large_audio_transcription_on_silence(path):
    """Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks"""
    try:
        # open the audio file using pydub
        sound = AudioSegment.from_file(path)  
        # split audio sound where silence is 500 miliseconds or more and get chunks
        chunks = split_on_silence(sound,
            # experiment with this value for your target audio file
            min_silence_len = 500,
            # adjust this per requirement
            silence_thresh = sound.dBFS-14,
            # keep the silence for 1 second, adjustable as well
            keep_silence=500,
        )
        folder_name = "audio-chunks"
        # create a directory to store the audio chunks
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
        whole_text = ""
        # process each chunk 
        for i, audio_chunk in enumerate(chunks, start=1):
            # export audio chunk and save it in
            # the `folder_name` directory.
            chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
            audio_chunk.export(chunk_filename, format="wav")
            # recognize the chunk
            try:
                text = transcribe_audio(chunk_filename)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
        # return the text for all chunks detected
        return whole_text
    except:
        return "Some Error!"


download_status = yt_download_video(link)
if download_status == "Task Completed!":
    vf = link.split("=")[-1]+".webm"
    audio_status = convert_video_to_audio_moviepy(vf)
else:
    print(download_status)

if audio_status == "Task Completed!":
    path = link.split("=")[-1]+".wav"
    transcribe_text = get_large_audio_transcription_on_silence(path)
else:
    print(download_status)

print(transcribe_text)
